In [13]:
import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data', one_hot = True)

Extracting /tmp/data\train-images-idx3-ubyte.gz
Extracting /tmp/data\train-labels-idx1-ubyte.gz
Extracting /tmp/data\t10k-images-idx3-ubyte.gz
Extracting /tmp/data\t10k-labels-idx1-ubyte.gz


In [111]:
tf.reset_default_graph()

learning_rate = 0.01
dropout_input = 0.8
dropout_hidden = 0.8
batch_size = 256
num_steps = 7000
display_steps = 200
beta = 0

num_input = 28*28
num_hidden1 = 500
num_hidden2 = 500
num_hidden3 = 500
num_layers = 3
num_classes = 10

X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])

In [112]:
def neural_net(x, weights, biases, dropout_input, dropout_hidden):
    #reshape x back to be an image
    #x = tf.reshape(x, [-1, 28, 28 ,1])
    
    layer1 = tf.matmul(x, weights['w1']) + biases['b1']
    layer1 = tf.nn.dropout(tf.nn.relu(layer1), dropout_input)
    
    layer2 = tf.matmul(layer1, weights['w2']) + biases['b2']
    layer2 = tf.nn.dropout(tf.nn.relu(layer2), dropout_hidden)
    
    layer3 = tf.matmul(layer2, weights['w3']) + biases['b3']
    layer3 = tf.nn.dropout(tf.nn.relu(layer3), dropout_hidden)
    
    output = tf.matmul(layer3, weights['w_out']) + biases['b_out']
    return output

In [113]:
weights = {
    'w1': tf.Variable(tf.random_normal([num_input, num_hidden1])),
    'w2': tf.Variable(tf.random_normal([num_hidden1, num_hidden2])),
    'w3': tf.Variable(tf.random_normal([num_hidden2, num_hidden3])),
    'w_out': tf.Variable(tf.random_normal([num_hidden3, num_classes])),
}

biases = {
    'b1': tf.Variable(tf.random_normal([num_hidden1])),
    'b2': tf.Variable(tf.random_normal([num_hidden2])),
    'b3': tf.Variable(tf.random_normal([num_hidden3])),
    'b_out': tf.Variable(tf.random_normal([num_classes])),
}

logits = neural_net(X, weights, biases, dropout_input, dropout_hidden)

regularizer = tf.nn.l2_loss(weights['w1']) + tf.nn.l2_loss(weights['w2']) + tf.nn.l2_loss(weights['w_out'])
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = Y) + beta * regularizer)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
train_op = optimizer.minimize(loss_op)

correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

In [114]:
with tf.Session() as sess:
    sess.run(init)
    
    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        
        sess.run(train_op, feed_dict = {X: batch_x, Y:batch_y})
        
        if step % display_steps == 0 or step == 1:
            loss, acc = sess.run([loss_op, accuracy], feed_dict = {X: batch_x, Y:batch_y})
            print("Step" + str(step) + ", Minibatch loss" +"{:.4f}".format(loss) + ", Trainning accuracy" + "{:.3f}".format(acc))
    
    print("Optimization done!")
    
    test_accu = sess.run(accuracy, feed_dict = {X: mnist.test.images, Y:mnist.test.labels})
    
    print("Test accuracy:", test_accu)

Step1, Minibatch loss51057.6836, Trainning accuracy0.145
Step200, Minibatch loss883.1470, Trainning accuracy0.879
Step400, Minibatch loss385.4158, Trainning accuracy0.891
Step600, Minibatch loss219.8822, Trainning accuracy0.895
Step800, Minibatch loss98.0753, Trainning accuracy0.898
Step1000, Minibatch loss145.4314, Trainning accuracy0.875
Step1200, Minibatch loss54.8341, Trainning accuracy0.883
Step1400, Minibatch loss47.9446, Trainning accuracy0.801
Step1600, Minibatch loss22.1117, Trainning accuracy0.637
Step1800, Minibatch loss4.8061, Trainning accuracy0.488
Step2000, Minibatch loss2.7673, Trainning accuracy0.352
Step2200, Minibatch loss3.0988, Trainning accuracy0.293
Step2400, Minibatch loss4.3245, Trainning accuracy0.336
Step2600, Minibatch loss2.1627, Trainning accuracy0.316
Step2800, Minibatch loss2.0585, Trainning accuracy0.320
Step3000, Minibatch loss2.4539, Trainning accuracy0.266
Step3200, Minibatch loss1.7789, Trainning accuracy0.301
Step3400, Minibatch loss1.7991, Trainni